In [1]:
#Read in Lexical/citation_scraper pmids
#path = "/Users/manatee_prime/Documents/covid/"
path = "C:\\Users\\sean.klein\\Documents\\Research\\LnkRsrchPol\\"

ncas = [[],[]] #0 = NCA name, 1 = NCA publication date
pmids = []

f = open(path + 'lexical_pmids.csv','r')
for line in f:
    pmids.append(line.split(',')[-1].rstrip())
    ncas[0].append(line.split(',')[0].strip().replace('\"',''))
    ncas[1].append(line.split(',')[1].strip())
del pmids[0] #get rid of column names by deleting first position in each list
del ncas[0][0]
del ncas[1][0]
print(len(pmids),pmids[0:3],ncas[0][0:3],ncas[1][0:3])

7029 ['54679', '121949', '383946'] ['Proposed Decision Memo for Erythropoiesis Stimulating Agents (ESAs) for Treatment of Anemia in Adults with CKD Including Patients on Dialysis and Patients not on Dialysis (CAG-00413N)', 'Decision Memo for Blood Brain Barrier Disruption (BBBD) Chemotherapy (CAG-00333N)', 'Decision Memo for Gender Dysphoria and Gender Reassignment Surgery (CAG-00446N)'] ['2011-03-16', '2007-03-20', '2016-08-30']


In [2]:
#Read in pickled lists 

import pickle

#path = "/Users/manatee_prime/Documents/covid/"
path = "C:\\Users\\sean.klein\\Documents\\Research\\LnkRsrchPol\\"

with open(path + "lexical_article_pickle.txt", "rb") as fp:   # Unpickling
    article = pickle.load(fp)
with open(path + "lexical_fgrant_pickle.txt", "rb") as fp:   # Unpickling
    fgrant = pickle.load(fp)

In [ ]:
#This code returns grant information on all pmids gathered from pubmed
#It is modified slightly to retrieve additional info (article name, author affiliation)
#NOTE: There is an issue with the journal scraping as we aren't catching something

import collections
import requests
import urllib.request
from bs4 import BeautifulSoup
import time
import pickle

#NOTE: consider adding article date information to grant scraper for accurate dating down to the day
#      use 'pubdate' to grab date information for future grant searches

fgrant = [[],[],[]] #0 = grant num, 1 = grant agency, 2 = grant country
article = [] #0 = pub date, 1 = journal, 2 = title, 3 = article type (letter, research art, rev, etc.)
ct = 0

for x in pmids:
    
    #Counter to track number of citations tackled
    ct += 1
    if ct % 300 == 0:
        print(ct/len(pmids),' completed!')
    
    skey = 'dcff69acdc62008c90b1354b0ac141dc0808' #api key to get more queries to ncbi eutils
    time.sleep(0.2) #setting to 0.1 throws over-query error
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={0}&retmode=xml&api_key={1}".format(x,skey)
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    
    ###SCRAPE PUBLICATION ARTICLE INFORMATION###
    #Date
    datetst = True
    date = soup.find('pubmedpubdate',attrs={'pubstatus':"accepted"})
    #print(response,'\n',date,'\n')
    if date == None: #Article information not listed in pubmed and need to skip this pmid
        date = soup.find('pubmedpubdate',attrs={'pubstatus':"pubmed"})
        if date == None:
            article.append('NA')
            datetst = False
    
    pubdate = ''
    if datetst == True:
        for item in date:
            for subitem in item: #pubdate lists in order: year, month, day
                try:
                    int(subitem)
                    if pubdate.count('-') == 2:
                        break
                    if pubdate != '':
                        pubdate = pubdate + '-' + str(subitem)
                    if pubdate == '':
                        pubdate = str(subitem)
                except ValueError:
                    continue
        article.append(pubdate)
    
    ###SCRAPE GRANT INFORMATION###
    content = soup.find_all('grant')
    if len(content) == 0:
        fgrant[0].append(['NA']) #seems weird to append list but when grant info exists, it is appended as list
        fgrant[1].append(['NA']) #so this maintains that list of lists structure
        fgrant[2].append(['NA'])
        continue

    grant = [[],[],[]]
    for item in content:
        gid = False
        agency = False
        country = False
            
        for j in item:
            if 'grantid' in str(j):
                grant[0].append(str(j).split('>')[1].split('<')[0])
                gid = True
                continue
            if 'agency' in str(j):
                grant[1].append(str(j).split('>')[1].split('<')[0])
                agency = True
                continue
            if 'country' in str(j):
                grant[2].append(str(j).split('>')[1].split('<')[0])
                country = True
                continue
            
        if gid == False:
            grant[0].append('NA') #DO NOT append as list as these are items within a list of grant data already!
        if agency == False:
            grant[1].append('NA')
        if country == False:
            grant[2].append('NA')
    fgrant[0].append(grant[0])
    fgrant[1].append(grant[1])
    fgrant[2].append(grant[2])
    ###END GRANT BLOCK###

###SAVE LISTS TO PICKLE IN CASE LOSE KERNEL###        
with open(path + "lexical_article_pickle.txt", "wb") as fp:   #Pickling
    pickle.dump(article, fp)
with open(path + "lexical_fgrant_pickle.txt", "wb") as fp:   #Pickling
    pickle.dump(fgrant, fp)

print('article lengths ',len(article))
print('fgrant lengths ',len(fgrant[0]),len(fgrant[1]),len(fgrant[2]))

In [ ]:
#Simple evaluation stats for grant data
import collections

a = [x for item in fgrant[1] for x in item]
agencies = collections.Counter(a)
print(agencies.most_common(11)) #top 10 non-NA results as 'NA' is among top results

In [18]:
#Create nodes and edges file for visNetwork in R
#Nodes file == list of all possible nodes
#Edges file == [[from],[to],[weight]], for this analysis weight == 1 unless to CMS, then it == counts
#NOTE - Have removed redundant citations. Could track weighting of pmids (# occurences in nca corpus) but easier to
#       do that by looking directly at pmids list so implement weighting in code above!

import collections

uid = collections.Counter(pmids)
tmpl2 = [] #this will house all the combinations of grants
weight = [] #for tracking # times a pmid occurs in nca corpus
#edges = [[],[],[]] #0 = origin node, 1 = destination node, 2 = edge weight

for num in uid:
    tmpl = [] #re-create tmp list for each iteration
    
    redund = False #to prevent appending multiple pmid hits and overcounting
    for i in range(len(pmids)):
        if pmids[i] == num and fgrant[1][i] != ['NA'] and redund == False: #no need to do this with NA-only entries
            tmpl.append(fgrant[1][i])
            redund = True
    
    if len(tmpl) == 0: #move on from missing grant data
        continue
    if len(tmpl[0]) == 1: #ensure singletons still count towards CMS, need [0] index as will be list of lists
        tmpl2.append(tmpl[0][0]+'_CMS')
        continue
    
    #join all grants funding same publication
    tmpl = [y for x in tmpl for y in x] #flatten tmpl into list
    tmpl = sorted(tmpl) #arrange alphabetically to impose standard order
    if 'NCI NIH HHS' in tmpl:
        print(tmpl,num)
    
    for j in range(len(tmpl)):
        tmpl2.append(tmpl[j]+'_CMS') #each grant in tmpl[0] connects to CMS
        for k in range(j+1,len(tmpl)):
            tmpl2.append(tmpl[j]+'_'+tmpl[k]) #each grant within a pub should connect to eachother

edgect = collections.Counter(tmpl2)

['NCI NIH HHS', 'NCI NIH HHS'] 1342322
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 1988575
['NCI NIH HHS', 'NIGMS NIH HHS', 'NIGMS NIH HHS'] 2841597
['NCI NIH HHS', 'NIDDK NIH HHS'] 2910371
['NCI NIH HHS', 'NEI NIH HHS'] 3278233
['NCI NIH HHS', 'NCI NIH HHS'] 3756785
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 6090973
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 6251399
['NCI NIH HHS', 'NCI NIH HHS'] 6410302
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 6782926
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 6895614
['NCI NIH HHS', 'NIADDK NIH HHS'] 7204572
['NCI NIH HHS', 'NHLBI NIH HHS', 'NHLBI NIH HHS'] 7276829
['NCI NIH HHS', 'NCI NIH HHS'] 8400304
['NCI NIH HHS', 'NCRR NIH HHS'] 8010342
['NCI NIH HHS', 'NCI NIH HHS'] 8201388
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 8474513
['NCI NIH HHS', 'NCI NIH HHS'] 8757769
['NCI NIH HHS', 'NCI NIH HHS', 'NHLBI NIH HHS'] 9118298
['NCI NIH HHS', 'NCI NIH HHS'] 9313876
['NCI NIH HHS', 'NCI NIH HHS'] 9607579
['NCI NIH HHS', 'NINDS NIH HHS

['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 29961768
['NCI NIH HHS', 'NIDDK NIH HHS'] 8343951
['NCI NIH HHS', 'NCI NIH HHS'] 17345612
['NCI NIH HHS', 'NCI NIH HHS', 'PHS HHS'] 18617085
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 15385124
['NCI NIH HHS', 'NIAID NIH HHS'] 8815760
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 9769331
['NCI NIH HHS', 'NCI NIH HHS'] 7028788
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 22889774
['NCI NIH HHS', 'NCI NIH HHS'] 17470865
['NCI NIH HHS', 'NINDS NIH HHS', 'NINDS NIH HHS'] 10649259
['NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS', 'NCI NIH HHS'] 12499279
['NCI NIH HHS', 'NCI NIH HHS'] 3112602
[

In [20]:
import csv
#path = "/Users/manatee_prime/Desktop/NCA/"
path = "C:\\Users\\sean.klein\\Documents\\Research\\LnkRsrchPol\\"

#write out raw grant info file
with open(path + 'NCA_grants.csv', 'w', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['NCA','NCA Date','PMID','Pub Date','Grant Number','Grant Agency','Agency Country']) #set up to match visNetwork syntax
    for i in range(len(fgrant[1])):
        for j in range(len(fgrant[1][i])): #need a separate line for each grant, even if duplicating pmid, nca, etc.
            writer.writerow([ncas[0][i],ncas[1][i],pmids[i],article[i],fgrant[0][i][j],fgrant[1][i][j],fgrant[2][i][j]])
csvfile.close()

#write out edges file
with open(path + 'NCA_grantagency_ntwk_edges.csv', 'w', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['from','to','value']) #set up to match visNetwork syntax
    for item in edgect:
        writer.writerow([item.split('_')[0],item.split('_')[1],edgect[item]])
csvfile.close()

#nodes can be created by using the unique() function in R on the edges file

In [19]:
edgect.most_common(20)

[('NCI NIH HHS_NCI NIH HHS', 4016),
 ('NHLBI NIH HHS_NHLBI NIH HHS', 2684),
 ('NIA NIH HHS_NIA NIH HHS', 2466),
 ('NIAID NIH HHS_NIAID NIH HHS', 1778),
 ('NCI NIH HHS_CMS', 717),
 ('NHLBI NIH HHS_CMS', 663),
 ('NIAID NIH HHS_NIDA NIH HHS', 644),
 ('NIDDK NIH HHS_NIDDK NIH HHS', 522),
 ('NIA NIH HHS_CMS', 518),
 ('NCATS NIH HHS_NCATS NIH HHS', 427),
 ('NIAAA NIH HHS_NIAID NIH HHS', 332),
 ('NCRR NIH HHS_NIAID NIH HHS', 275),
 ('NCATS NIH HHS_NCRR NIH HHS', 249),
 ('NIDDK NIH HHS_CMS', 237),
 ('NIAMS NIH HHS_NIAMS NIH HHS', 225),
 ('NIAID NIH HHS_NIMH NIH HHS', 208),
 ('NCI NIH HHS_NHLBI NIH HHS', 201),
 ('NHLBI NIH HHS_NIGMS NIH HHS', 190),
 ('NCATS NIH HHS_NHLBI NIH HHS', 186),
 ('NIA NIH HHS_NIMH NIH HHS', 169)]